In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy

from astropy.io import fits
from astropy.table import Table

In [4]:
def string_transfer(sexa):
    """Convert sexagesimal string like '12:01:01.5' or '-12:01:01.5' 
    into (sign, [degree/hour, minute, second])."""
    sign = True
    if sexa.startswith("-"):
        sign = False
        sexa = sexa[1:]
    elif sexa.startswith("+"):
        sexa = sexa[1:]
    
    parts = sexa.split(":")
    if len(parts) != 3:
        raise ValueError(f"Invalid sexagesimal string: {sexa}")

    deg_or_hour = float(parts[0])
    minute = float(parts[1])
    second = float(parts[2])
    
    return sign, [deg_or_hour, minute, second]


def h_to_decimal(h):
    """RA: hh:mm:ss.s -> degrees"""
    hour, minute, second = h
    result = hour * 15 + minute * (15/60) + second * (15/3600)
    return result


def d_to_decimal(d):
    """Dec: dd:mm:ss.s -> degrees"""
    degree, minute, second = d
    result = degree + minute * (1/60) + second * (1/3600)
    return result


def sexa_to_decimal(RA, Dec):
    """Convert RA and Dec strings to decimal degrees."""
    _, RA_list = string_transfer(RA)
    sign, Dec_list = string_transfer(Dec)

    RA_r = h_to_decimal(RA_list)
    Dec_r = d_to_decimal(Dec_list)

    print('R.A.:', RA_r)
    if sign:
        print('Dec.:', '+', Dec_r)
    else:
        print('Dec.:', '-', Dec_r)

    return RA_r, Dec_r


if __name__ == "__main__":
    Sample_RA = "12:01:01.5"
    Sample_Dec = "-12:01:01.5"
    sexa_to_decimal(Sample_RA, Sample_Dec)

    
    

R.A.: 180.25625
Dec.: - 12.017083333333334


In [2]:
# The Region File

file = '/Users/wanyan/Desktop/Astro/229/229_data/stars_near_M82.txt'
def is_valid_line(line):
    if line.strip() == "":
        return False
    if line.strip().startswith("#"):
        return False

    parts = [p.strip() for p in line.split(",")]
    if len(parts) < 3:
        return False

    coords = parts[2].split()
    if len(coords) < 6:
        return False
    return True

def coord_transfer(coord):
    temp = coord.strip().split()
    out = [0] * 3
    out[0] = int(temp[0])       
    out[1] = int(temp[1])
    out[2] = float(temp[2])
    return out

def line_to_region(line, radius='10"'):
    parts = [p.strip() for p in line.split(",")]
    coords = parts[2].split()

    RA = coord_transfer(" ".join(coords[0:3]))
    Dec = coord_transfer(" ".join(coords[3:6]))

    RA_s = f"{RA[0]:02d}:{RA[1]:02d}:{RA[2]:05.2f}"
    Dec_s = f"{Dec[0]:+d}:{Dec[1]:02d}:{Dec[2]:05.2f}"   

    return f"circle {RA_s} {Dec_s} {radius}"

if __name__ == "__main__":
    infile = "/Users/wanyan/Desktop/Astro/229/229_data/stars_near_M82.txt"
    outfile = "Regions_M82.reg"

    regions = []
    with open(infile, "r") as f:
        lines = f.readlines()
        for line in lines:
            if not is_valid_line(line):
                continue
            regline = line_to_region(line)
            if regline:
                regions.append(regline)

    with open(outfile, "w") as f:
        f.write("fk5\n")
        for reg in regions:
            f.write(reg + "\n")